In [49]:
AREAS = [
'Semantic segmentation',
'Image classification',
'Object detection',
'Object Recognition',
'Domain adaptation',
'Image generation',
'Image Captioning',
'Image augmentation',
'Pose estimation',
'Autonomous vehicles',
'Denoising',
'Super-Resolution',
'Object Tracking',
'Action Recognition',
'Face Recognition',
'Depth Estimation',
'Optical Character Recognition',
'3D Reconstruction',
'Image Retrieval',
'Optical Flow Estimation',
'Style Transfer',
'Image Compression']

# Google Scholar Crawl

In [50]:
import pandas as pd
from serpapi import GoogleSearch
from tqdm import tqdm

In [52]:
gs_raw = {}
total_paper = 400
for area in tqdm(AREAS):
    area_raw = []
    for ii in range(total_paper//20):
        params = {
        'api_key': '9cdc23221a6d7cce098cebc84adb8ab98cb53e8870970758225b506e8cb33757',
        'engine': 'google_scholar',
        'q': f'{area}',
        'hl': 'en',
        'as_ylo': '2000',
        'num': '20',
        'start': f'{20*ii}'
        }
        search = GoogleSearch(params)
        area_raw += search.get_dict().get('organic_results')
    gs_raw[area] = area_raw

100%|██████████| 22/22 [12:59<00:00, 35.44s/it]


In [61]:
gs_results = []
keys_to_extract = ['title', 'link', 'snippet']
for area in AREAS:
    for rank, paper in enumerate(gs_raw[area]):
        paper_result = {'area': area, 'rank': rank + 1}
        for key in keys_to_extract:
            paper_result[key] = paper.get(key)
        if 'resources' in paper:
            for resource in paper['resources']:
                if resource.get('file_format') == 'PDF' and resource['link']:
                    paper_result['pdf_link'] = resource['link']
                    break
        if 'pdf_link' not in paper_result:
            paper_result['pdf_link'] = None
        
        gs_results.append(paper_result)
    
    

In [62]:
gs_df = pd.DataFrame(gs_results)
gs_df.to_csv('./data/gs_results.csv', index=False)

# Semantic Segmenation

In [63]:
import requests

In [64]:
ss_results = []
for area in tqdm(AREAS):
    raw = requests.get('https://api.semanticscholar.org/graph/v1/paper/search?query={}&limit=100&fields=url,title,abstract,referenceCount,citationCount,influentialCitationCount,isOpenAccess,fieldsOfStudy'.format('+'.join(area.lower().split(' ')))).json().get('data')
    raw += requests.get('https://api.semanticscholar.org/graph/v1/paper/search?query={}&limit=100&offset=100&fields=url,title,abstract,referenceCount,citationCount,influentialCitationCount,isOpenAccess,fieldsOfStudy'.format('+'.join(area.lower().split(' ')))).json().get('data')
    for rank, paper in enumerate(raw):
        paper['area'] = area
        paper['rank'] = rank + 1
        ss_results.append(paper)

100%|██████████| 22/22 [00:46<00:00,  2.10s/it]


In [65]:
ss_df = pd.DataFrame(ss_results)
ss_df.dropna()
ss_df = ss_df[['area','rank','title','referenceCount', 'url', 'citationCount', 'influentialCitationCount', 'isOpenAccess', 'fieldsOfStudy', 'abstract']]

In [66]:
ss_df.to_csv('./data/ss_results.csv', index=False)

# Paperwithcodes

In [67]:
from bs4 import BeautifulSoup

In [68]:
total_paper = 200
pc_results = []
for area in tqdm(AREAS):
    area_formated = '-'.join(area.lower().split(' '))
    for ii in range(total_paper//10):
        html_raw = requests.get(f'https://paperswithcode.com/task/{area_formated}?page={ii+1}')
        soup = BeautifulSoup(html_raw.text, 'html.parser')
        paper_cards = soup.find_all('div', class_='paper-card infinite-item')
        for jj, paper_card in enumerate(paper_cards):
            rank = ii * 10 + jj + 1
            result = {'area': area, 'rank': rank}
            result['title'] = paper_card.find('h1').text
            pc_results.append(result)

100%|██████████| 22/22 [08:40<00:00, 23.67s/it]


In [71]:
pc_df = pd.DataFrame(pc_results)
pc_df.to_csv('./data/pc_results.csv', index=False)

In [72]:
areas = pc_df['area'].unique().tolist()
for area in AREAS:
    if area not in areas:
        print(area)